In [77]:
!pip install yfinance tensorflow scikit-learn matplotlib

In [78]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import warnings

In [88]:
# Fetch Bajaj Auto data from Yahoo Finance (symbol: BAJAJ-AUTO.NS for NSE India)
df = yf.download("ETERNAL.NS", start="2020-01-01", end="2025-09-06")
df = df[['Close']]
df.tail()

C:\Users\jyks\AppData\Local\Temp\ipykernel_15788\1471844947.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download("ETERNAL.NS", start="2020-01-01", end="2025-09-06")
[*********************100%***********************]  1 of 1 completed


Price,Close
Ticker,ETERNAL.NS
Date,
2025-09-01,321.100006
2025-09-02,322.399994
2025-09-03,326.100006
2025-09-04,326.250000
2025-09-05,329.149994


In [95]:
# Normalize data
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(df)

# Training/test split
train_size = int(len(scaled_data) * 0.8)
train_data = scaled_data[:train_size]
test_data = scaled_data[train_size:]


In [96]:
def create_sequences(data, seq_length=60):
    X, y = [], []
    
    # Check if data is 1D or 2D
    if len(data.shape) == 1:
        # For 1D data
        for i in range(seq_length, len(data)):
            X.append(data[i-seq_length:i])
            y.append(data[i])
    else:
        # For 2D data
        for i in range(seq_length, len(data)):
            X.append(data[i-seq_length:i, 0])
            y.append(data[i, 0])
            
    return np.array(X), np.array(y)

seq_length = 60
X_train, y_train = create_sequences(train_data, seq_length)
X_test, y_test = create_sequences(test_data, seq_length)

# Add check to ensure we have data before reshaping
if len(X_train) > 0 and len(X_test) > 0:
    # Reshape for LSTM [samples, time_steps, features]
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
else:
    print("Warning: Empty arrays detected. Check your data and sequence length.")
    # Initialize with proper shape but empty arrays if needed
    if len(X_train) == 0:
        X_train = np.empty((0, seq_length, 1))
    if len(X_test) == 0:
        X_test = np.empty((0, seq_length, 1))

In [97]:
model = Sequential()
model.add(LSTM(100, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(100, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(50))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1)


Epoch 1/20


C:\Users\jyks\Anacondav3\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - loss: 0.4508 - val_loss: 0.4341
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step - loss: 0.2820 - val_loss: 0.2248
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step - loss: 0.1538 - val_loss: 0.0732
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step - loss: 0.0627 - val_loss: 0.0013
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step - loss: 0.0218 - val_loss: 0.0679
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step - loss: 0.0503 - val_loss: 0.1176
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step - loss: 0.0870 - val_loss: 0.0710
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step - loss: 0.0589 - val_loss: 0.0210
Epoch 9/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step - loss: 0.0272 - val_loss: 0.0015
Epoch 10/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step - loss: 0.0251 - val_loss: 0.0039
Epoch 11/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step - loss: 0.0251 - val_loss: 0.0131
Epoch 12/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step - loss: 0.0302 - val_loss: 0.0214
Epo

In [104]:
import numpy as np
import matplotlib.pyplot as plt

# Make predictions
predictions = model.predict(X_test)

# Check for and handle any NaN or infinite values before inverse transform
predictions = np.nan_to_num(predictions)
predictions = scaler.inverse_transform(predictions)

# Ensure y_test is properly shaped and handle any NaN values
actual = y_test.reshape(-1, 1)
actual = np.nan_to_num(actual)
actual = scaler.inverse_transform(actual)

# Additional safety: replace any remaining inf values after inverse_transform
predictions = np.clip(predictions, -1e10, 1e10)  # Clip extreme values
actual = np.clip(actual, -1e10, 1e10)  # Clip extreme values

# Plot with error handling
plt.figure(figsize=(10, 6))
plt.plot(actual, label="Actual Price")
plt.plot(predictions, label="Predicted Price")
plt.legend()

# Set axis limits more safely with additional checks
# Use masked arrays to ignore NaN and inf values
masked_actual = np.ma.masked_invalid(actual)
masked_predictions = np.ma.masked_invalid(predictions)

try:
    # Only proceed if we have valid data points
    if masked_actual.count() > 0 and masked_predictions.count() > 0:
        min_val = min(np.ma.min(masked_actual), np.ma.min(masked_predictions))
        max_val = max(np.ma.max(masked_actual), np.ma.max(masked_predictions))
        
        # Additional check to ensure values are finite
        if np.isfinite(min_val) and np.isfinite(max_val):
            plt.ylim(bottom=min_val * 0.9, top=max_val * 1.1)
        else:
            plt.autoscale(enable=True, axis='y')
    else:
        plt.autoscale(enable=True, axis='y')
except:
    # Fallback if anything goes wrong with the calculations
    plt.autoscale(enable=True, axis='y')

plt.show()

ValueError: math domain error

In [ ]:
last_sequence = test_data[-seq_length:]  # last 60 days
forecast = []

current_seq = last_sequence.reshape(1, seq_length, 1)

for i in range(14):  # next 7 days
    pred = model.predict(current_seq)[0][0]
    forecast.append(pred)

    # update sequence
    current_seq = np.append(current_seq[:,1:,:], [[[pred]]], axis=1)

forecast_prices = scaler.inverse_transform(np.array(forecast).reshape(-1,1))
forecast_prices = forecast_prices.flatten()
forecast_prices


In [ ]:
last_date = df.index[-1]
future_dates = pd.date_range(last_date, periods=15, freq='B')[1:]  # business days

forecast_df = pd.DataFrame({
    "Date": future_dates,
    "Predicted Close (INR)": forecast_prices
})

print(forecast_df)
